## Description

#### Purpose: To engineer 'season' and 'holiday' variables and adjust budget/revenue figurs for inflation using CPI.

#### Input: `3.3.9b_Merged_Data_MPAA_Encoded.csv`

#### Outputs: `3.3.10_Merged_Data_Seasonality_Inflation.csv`

This notebook engineers variables capturing seasonality and then adjusts for inflation using a standard comparision of CPI to 1983.

The rationale for engineering features for seasonality is that May, June, and July are consistently the best months for a theatrical release while February, March, and April are dead months. Holiday releases also typically fare better. 

In [ ]:
from datetime import datetime
import pandas as pd

In [ ]:
def to_dt(release_date):
    try:
        date_l = release_date.split("(")[0]
        # Sample string representing a date
        date_string = date_l.strip()

        # Convert the string to a datetime object
        datetime_object = datetime.strptime(date_string, '%B %d, %Y')
        return datetime_object
    except:
        return pd.NaT

In [ ]:
df = pd.read_csv("./data_with_encoded_mpaa.csv")
df['US Release Date'] = pd.to_datetime(df['US Release Date'], errors='coerce')

In [ ]:
def split_by_month(date):
    try:
        if date.month in {5,6,7}:
            return "MJJ_3"
        elif date.month in {8,9,10}:
            return "ASO_4"
        elif date.month in {11,12,1}:
            return "NDJ_1"
        elif date.month in {2,3,4}:
            return "FMA_2"
    except:
        return pd.NA

In [ ]:
df["Season"] = df.apply(lambda x: split_by_month(x["US Release Date"]), axis=1)

In [ ]:
df["Season"]

In [ ]:
from datetime import datetime, timedelta, date
import holidays
def replace_date_with_datetime(input_dict):
    """
    Replace keys that are datetime.date objects with datetime.datetime objects.

    Parameters:
    input_dict (dict): The dictionary with datetime.date keys.

    Returns:
    dict: A new dictionary with datetime.datetime keys.
    """
    new_dict = {}
    for key, value in input_dict.items():
        # Check if the key is a datetime.date object and not a datetime object
        if isinstance(key, date) and not isinstance(key, datetime):
            # Convert to datetime.datetime
            new_key = datetime(key.year, key.month, key.day)
        else:
            # Keep the key as it is
            new_key = key
        new_dict[new_key] = value

    return new_dict

def is_near_holiday(input_date, country='US', days_before=7, days_after=5):
    """
    Check if a given date is within a week prior to or 5 days after a US holiday.

    Parameters:
    input_date (datetime): The date to check.
    country (str): The country code for the holidays. Default is 'US'.

    Returns:
    bool: True if the date is near a holiday, False otherwise.
    """
    us_holidays = holidays.CountryHoliday(country, years= [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023])
    us_holidays = replace_date_with_datetime(us_holidays)
    # Check a week before and 5 days after each holiday in the year of the input date
    for date in us_holidays.keys():
        if input_date.timestamp() >= (date - timedelta(days=days_before)).timestamp() and input_date.timestamp() <= (date + timedelta(days=days_after)).timestamp():
            return True

    return False

example_date = datetime(2020, 12, 25)  # New Year's Day in 2023
print(is_near_holiday(example_date))

In [ ]:
df["Holiday"] = df.apply(lambda x: is_near_holiday(x["US Release Date"]), axis=1)
df["Holiday"]

In [ ]:
df[df["Holiday"] == True]

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
import cpi
import numpy as np

def inflate(date, dollars):
    try:
        if date.year == 2023:
            multiplier = 100/302.9
            return dollars * multiplier
        else:
            return cpi.inflate(dollars, date.year, to=1983) #when cpi was 100
    except:
        return np.nan


In [ ]:
df["Adj Merged Budget"] = df.apply(lambda x: inflate(x["US Release Date"], x["Merged Budget"]), axis=1)
df["Adj Merged Revenue"] = df.apply(lambda x: inflate(x["US Release Date"], x["Merged Revenue"]), axis=1)
df[["IMDB ID", "Adj Merged Budget", "Merged Budget"]]

In [ ]:
df.to_csv('/Outputs/3.3.10_Merged_Data_Seasonality_Inflation.csv')